## Import Libraries

In [ ]:
# data analysis 
import pandas as pd
import numpy as np
import random as rnd
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import interp
from itertools import cycle
%matplotlib inline

# machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb


# Evaluating and tuning the models
from sklearn.model_selection import train_test_split , StratifiedKFold, GridSearchCV, KFold
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import label_binarize

## Load data

In [ ]:
# Acquire data
train_X = pd.read_csv('train_X.csv', sep=',')
train_Y = pd.read_csv('train_Y.csv', sep=';')
test_X = pd.read_csv('test_X.csv', sep=',')

## Statistical summaries and first observations

In [ ]:
# preview the data
train_X.head(3)

In [ ]:
train_Y.head(3)

In [ ]:
# Merge train_X and train_Y
train_df = pd.merge(train_X, train_Y, on='ID')
train_df.head(3)

In [ ]:
# ID feature is useless, we remove it from train set
train_df = train_df.drop(['ID'], axis=1)
train_df.head(3)

In [ ]:
# Define a list containing both train and test set to apply modifications on both sets in an iterative way
combine = [train_df, test_X]

In [ ]:
train_df.info()
print('_'*50,'\n')
test_X.info()

In [ ]:
train_df.describe()

In [ ]:
#Including only string columns
train_df.describe(include=[np.object])

In [ ]:
# Count number of null elements in a dataframe column
def count_null(col):
    return sum(col.isnull())

# Describe a dataframe column
def describe(col):
    description = col.value_counts()
    description['NULL']=count_null(col)
    print(description, '\n')

In [ ]:
# Description of each column
for col in train_df.columns.values:
    describe(train_df[col])

In [ ]:
# Percentage of claims
claims = train_df.CLAIM_TYPE.value_counts()
claims = round(claims.div(claims.sum()/100),1)
claims

##### Especially we can note that:

Most of the variables are Categorical:
* Nominal variables : 
    SHIPPING_MODE (11 levels)
    BUYER_DEPARTMENT (100 levels)
    BUYING_DATE (10 levels)
    SELLER_COUNTRY (39 levels)
    SELLER_DEPARTMENT (98 levels)
    PRODUCT_TYPE (137 levels)
    CLAIM_TYPE (8 levels)
* Dichotomous variables :
    WARRANTIES_FLG
    CARD_PAYMENT
    COUPON_PAYMENT
    RSP_PAYMENT
    WALLET_PAYMENT
* Ordinal variables :
    SHIPPING_PRICE (5 levels)
    WARRANTIES_PRICE (5 levels)
    PRICECLUB_STATUS (5 levels)
    PURCHASE_COUNT (6 levels)
    SELLER_SCORE_COUNT (5 levels)
    ITEM_PRICE (8 levels)
    
* Quantitative variables : 
    REGISTRATION_DATE (17 distinct values)
    BUYER_BIRTHDAY_DATE (107 distincts values)
    SELLER_SCORE_AVERAGE (31 distincts values)
    
Quantitative variables could also be considered as Categorical

Some variables have many missing values:
* WARRANTIES_PRICE : 96603 / 96.6% missing values
* SHIPPING_PRICE : 67610 / 67.6% missing values
* BUYER_BIRTHDAY_DATE : 5836 / 5.8% missing values

Dichotomous classes are not well balanced (thus the use of AUC weighted metric):
* WARRANTIES_FLG : 96603 / 96.6% zeros
* COUPON_PAYMENT : 94271 / 94.3% zeros
* CARD_PAYMENT : 10407 / 89,6% ones
* WALLET_PAYMENT : 87045 / 87.0% zeros
* RSP_PAYMENT : 82942 / 82.9% zeros
* CLAIM_TYPE : 49977 / 50.0% '-' (no claim)

Some categorical variables have a high cardinality:
* PRODUCT_TYPE : 137 distinct values
* BUYER_DEPARTMENT 100 distinct values
* SELLER_DEPARTMENT 98 distinct values
* SELLER_COUNTRY 39 distinct values

Dataset has 3238 duplicate rows: Some users may appear in several rows (cf introduction video).
Some numerical values are discretized for anonymisation puropose but it might be possible to identify individuals.

Features are quite heterogeneous and complex.
Thus it could be interesting to find new socio-demographic features based on the current ones to feed our model.

## In-depth dataset study

In this section we will study the dataset in greater details. This will help creating new variables / features to train the classifiers (feature engineering).

In [ ]:
# Percentage of claims for each level of categorical variable col, sorted by level importance
def claim_percentage_crosstab(col):
    df = pd.crosstab(train_df[col], train_df.CLAIM_TYPE).sort_values(by=['-'], ascending=False)
    return round(df.div(df.sum(axis=1)/100, axis=0),1)

# Plot correlation map given a dataframe df
# Note: works only on quantitative variables
def plot_correlation_map(df):
    corr = df.corr()
    _, ax = plt.subplots(figsize=(12, 10))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={'shrink':.9}, 
        ax=ax, 
        annot = True, 
        annot_kws = {'fontsize':12})

In [ ]:
# A heat map of correlation may give us a understanding of which variables are important
plot_correlation_map(train_df)

#### SHIPPING_MODE and SHIPPING_PRICE :

In [ ]:
# Percentage of each claim given SHIPPING_MODE
claim_percentage_crosstab('SHIPPING_MODE')

In [ ]:
# Percentage of each claim given SHIPPING_PRICE
claim_percentage_crosstab('SHIPPING_PRICE')

In [ ]:
# Link between SHIPPING_PRICE and SHIPPING_MODE
pd.crosstab(train_df.SHIPPING_MODE, train_df.SHIPPING_PRICE)

Observations :

* The delivery ways which are not popular have in average more claims than others.
Especially for PICKUP with 38% of WITHDRAWAL claim.

* MONDIAL_RELAY_PREPAYE (54%), SUIVI (54%) and NORMAL (53%) have the greater percentage of regular deliveries (without claim), while RECOMMANDE (37%), PICKUP (26%) and Kiala (0%) have the greater percentage of claims.

* NORMAL delivery has the highest percentage of NOT_RECEIVED claims (16%) but the lowest percentage of WITHDRAWAL (4%) and UNDEFINED (2%) claims.

* SUIVI has the lowest percentage of NOT_RECEIVED mention among popular delivery ways

* CHRONOPOST has the highest percentage of DAMAGED claim

* SO_POINT_RELAIS, MONDIAL_RELAY and SO_RECOMMANDE has typically the same percentage of claims


* Paradoxically, claims percentage grows with the price of the delivery.
Especially : UNDEFINED and WITHDRAWAL claims are more likely to happen with a high price.


* Their is a clear correlation between SHIPPING_MODE and SHIPPING_PRICE.
For example SHIPPING_PRICE over 20 are more likely to be RECOMMANDE.
However the prices can vary for a given delivery way and the SHIPPING_PRICE must be kept into our model.

* As SHIPPING_MODE is not an ordinal variable we need to turn it into dummy variables

#### WARRANTIES_FLG and WARRANTIES_PRICE

In [ ]:
# Percentage of each claim given WARRANTIES_FLG
claim_percentage_crosstab('WARRANTIES_FLG')

In [ ]:
# Percentage of each claim given WARRANTIES_PRICE
claim_percentage_crosstab('WARRANTIES_PRICE')

Observations :

* People having a warranty are slightly more likely to formulate a claim
Especially : WITHDRAWAL happen more frequently when a Warranty is taken.
This make sense since withdrawal possiblity can be included in a warranty

* Their is no clear link between warranties prices and claims

* As the number of null values for WARRANTIES_PRICE is equal to the number of False values for WARRANTIES_FLG, null values exactly correspond to the absence of warranty and can be put at 0 during mapping.

#### PRICECLUB_STATUS

In [ ]:
# Percentage of each claim given PRICECLUB_STATUS
claim_percentage_crosstab('PRICECLUB_STATUS').reindex(['UNSUBSCRIBED', 'REGULAR', 'PLATINUM', 'SILVER', 'GOLD'])

Observations:

* PRICECLUB_STATUS are linked to a number of accumulated points won when doing actions as:
selling products, refering a friend, using the prime minister application ...
With these points the customer can occasionally benefit from free purchases and gifts

* There is no clear link between the PRICECLUB_STATUS and claims.

* As UNSUBSCRIBED level represent over 60% of PRICECLUB_STATUS values, null values can be put at the same value than UNSUBSCRIBED during the mapping.

* PRICECLUB_STATUS can be considered as an ordinal variable since their is a rank between the types of status:
UNSUBSCRIBED<REGULAR<PLATINUM<SILVER<GOLD.

#### REGISTRATION_DATE and PURCHASE_COUNT

In [ ]:
# Percentage of each claim given REGISTRATION_DATE
new_index = [i for i in range(2001,2018)]
df = claim_percentage_crosstab('REGISTRATION_DATE').reindex(new_index)
plot = df.plot(figsize=(10, 6));
plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
# Percentage of each claim given PURCHASE_COUNT
new_index = ['<5','5<20','20<50','50<100','100<500','>500']
claim_percentage_crosstab('PURCHASE_COUNT').reindex(new_index)

In [ ]:
# Link between PURCHASE_COUNT and REGISTRATION_DATE
pd.crosstab(train_df.REGISTRATION_DATE, train_df.PURCHASE_COUNT).reindex(columns = new_index).plot(figsize=(10, 6));

Obervations:

* The different percentage for each type of claim given the date of registration are quite constant.
However, recent users tend to complain more often than old users as the level '-' is slightly decreasing over the year of registration

* It is noticeable that buyers with experience are less likely to have claim type within WITHDRAWAL and UNDEFINED.
Indeed a buyer with a great amount of commands is more susceptible to withdraw their command as they are accustomed to buying online.
However buyers with experience are also less suceptible to claim for damage or non reception of their command. This can be explained by the fact that these customers have their habits are may often pass command to seller which they already tried the effectiveness. Also Experience buyers may have good practices to avoid potential inneficient sellers that beginner custumer do not have. This feature will definitely be valuable for our model.

* As expected their is a link between these two features as buyers with a little number of commands are more suceptible to be recent users.

* Also, their is a clear gap between the most experienced users (<5 items) and other users. For this reason creating a new feature UNEXPERIENCED_USER could be valuable.

#### BUYER_BIRTHDAY_DATE

In [ ]:
# Percentage of each claim given BUYER_BIRTHDAY_DATE
new_index = [i for i in range(1902, 1981)]
plot = claim_percentage_crosstab('BUYER_BIRTHDAY_DATE').reindex(new_index).dropna()[30:].plot(); # Taking off extreme values
plot.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))

Observations :

* We can observe a correlation between age and claims as claims tend to increase while age decrease.
Especially young people seem more likely to claim NOT_RECEIVED which is not obvious.

* Replacing birhtday date by age would be simpler for our model

* There are a lot of Null values for this feature but it would not be efficient to use the mode or mean values in this case since levels are well balanced. The first option is to use a dynamic fill mathode as ffill which propagate last valid observation forward to next valid. The second option is to use knn on other features that may be correlated to BUYER_BIRTHDAY_DATE to fill the missing data.

#### BUYER_DEPARTMENT

To simplify the model we group buyers localisations by regions instead of departments.

In [ ]:
# Build a mapping dictionnary from departments to regions
DEPARTMENT_mapping = {}

Bretagne = [22,29,35,56]
Normandie = [14, 27, 50, 61, 76]
Hauts_De_France = [2,59,60,62,80]
Ile_De_France = [77,78,91,95]
Paris = [75,92,93,94]
Grand_Est =[8,10,51,52,54,55,57,67,68,88]
Bourgogne_France_Compte = [21,25,39,58,70,71,89,90]
Nouvelle_Aquitaine = [16,17,19,23,24,33,40,47,64,79,86,87]
Auvergne_Rhone_Alpes = [1,3,7,15,26,38,42,43,63,69,73,74]
Occitanie = [9,11,12,30,31,32,34,46,48,65,66,81,82]
Centre_Val_De_Loire = [18,28,36,37,41,45]
PACA = [4,5,6,13,83,84]
Pays_De_La_Loire = [44,49,53,72,85]
Etranger = [-1,20,97] # Putting out of France departments into Foreign list, 20=Corse, 97=DOM/TOM
Null = [0,96,98,99,100] # null values (department 97 and 98 do not exist)

Regions = [Bretagne,Normandie,Hauts_De_France,Ile_De_France,Paris,Grand_Est,Bourgogne_France_Compte,Nouvelle_Aquitaine,
           Auvergne_Rhone_Alpes,Occitanie,Centre_Val_De_Loire,PACA,Pays_De_La_Loire,Etranger,Null]

noms_Regions = ['Bretagne','Normandie','Hauts_De_France','Ile_De_France','Paris','Grand_Est','Bourgogne_France_Compte',
                'Nouvelle_Aquitaine','Auvergne_Rhone_Alpes','Occitanie','Centre_Val_De_Loire','PACA',
                'Pays_De_La_Loire','Etranger',0]

for ind, region in enumerate(Regions):
    for i in region:
        DEPARTMENT_mapping[i]=noms_Regions[ind]

# Creating BUYER_REGION :
for dataset in combine:
    dataset['BUYER_REGION'] = dataset['BUYER_DEPARTMENT'].map(DEPARTMENT_mapping)
    del dataset['BUYER_DEPARTMENT']

In [ ]:
# Percentage of each claim given BUYER_REGION
claim_percentage_crosstab('BUYER_REGION')

Observation : 

* Their is no clear correlation between BUYER_REGION and CLAIM_TYPE if buyer live in France.
We can only notice that buyer from Ile_De_France region (including Paris) and from PACA are slightly more suceptible to claim NOT_RECEIVED.

* However, as expected, claims happen more often abroad, as delivery is more complex.
Indeed commands passed abroad are more likely to receive claims NOT_RECEIVED or SELLER_CANCEL_POSTERIORI while on the contrary claims DAMAGED or DIFFERENT happen less frequently (this point is less obvious).

* Thus, it seems valuable to add a feature informing if the the command was passed in France or not.

* We could also turn BUYER_REGION into dummy variables to keep information about regions

#### BUYING_DATE

In [ ]:
# Mapping BUYING_DATE into numerical variable: '3/2017' -> 3
# Using regular expression to isolate months
for dataset in combine:
    dataset['BUYING_DATE'] = dataset['BUYING_DATE'].apply( lambda s : int(re.findall('[0-9]*',s)[0]) )

In [ ]:
# Percentage of each claim given BUYING_DATE
claim_percentage_crosstab('BUYING_DATE').reindex([i for i in range(1,11)])

In [ ]:
# frequency of commands for each month
train_df.BUYING_DATE.value_counts()

Observations:

* There is no clear correlation between BUYING_DATE and CLAIM_TYPE

* We can still notice that the month where the amount of commands is at the top (January) have the greatest percentage of claims while the month where the amount of commands is tIs he lowest (October) have the lowest percentage of claims.

* It is unfortunate that the data for November and December which are around Christmas are not available.
These data would have bring valuable information about a period that might be a peak period.

* With respect to the previous notes, keeping only 3 levels to separate the particualar months January and October from the rest of the months that have overall the same stats could represent a more valuable feature.

#### SELLER_SCORE_COUNT and SELLER_SCORE_AVERAGE

In [ ]:
# Percentage of each claim given BUYING_DATE
claim_percentage_crosstab('SELLER_SCORE_COUNT')

In [ ]:
# Percentage of each claim given SELLER_SCORE_AVERAGE
df=claim_percentage_crosstab('SELLER_SCORE_AVERAGE').reindex([i for i in range(50,40,-1)]) # Taking off extremely low values
df

In [ ]:
ax = df['-'].plot();
ax.set_ylabel("Percentage of delivery with no claim");

In [ ]:
# Link between SELLER_SCORE_COUNT and SELLER_SCORE_AVERAGE
pd.crosstab(train_df.SELLER_SCORE_COUNT, train_df.SELLER_SCORE_AVERAGE[train_df.SELLER_SCORE_AVERAGE>40] )

Observations :

* Logically we can observe that the amount of product sold by the seller is growing while the number of claims is decreasing. This is true for all the claim types except one : SELLER_CANCEL_POSTERIORI percentage tends to be higher for the range 100000<1000000 of SELLER_SCORE_COUNT.

* The null values of SELLER_SCORE_COUNT will be put at 0 in the mapping as it can correspond to seller without any product sold.

* Just as for the last feature it is clear that seller with the highest score are the more reliable and result in a fewer number of claims. Especially seller with a mark of 49 close to the maximum have a number of claims significatively low. Thus it could be useful to add a feature to discriminate these top seller.

* However an important point is that the best score possible 50 is reached only by a few individuals (51, while 3994 for 49 and 18006 for 48) and have in average in very bad reliability. We can also notice that these profiles all have sold less than 100 items. Thus, we can assume that some of these profiles are fake and manage somehow to get the maximum score to trick the system and get people trust. It could also be that those profiles have only a few sells hence the maximum score wich is impossible for a great number of sells.
Anyway, as a result these profiles will be put in the same level as seller with bad scores.

* We can add that without surprise their is an important correlation between SELLER_SCORE_AVERAGE and SELLER_SCORE_COUNT as reliable seller have in majority sold a lot of items.

#### SELLER_COUNTRY

In [ ]:
claim_percentage_crosstab('SELLER_COUNTRY')[:30] # Taking off country with a few number of deliveries

Observations:

* Most of European countries have correct percentage of delivery.
For instance: SWITZERLAND (50.1% no claim), GERMANY (60,9% no claim), BELGIUM (58,2% no claim), LUXEMBOURG (58,5% no claim).

* However some European country perform bad like UNITED KINGDOM	that have notably high percentage of NOT_RECEIVED claims (19.2%).

* It is hard to classify these country in classes as many countries have specificities (very high percentage of WITHDRAWAL claims for ITALY, overall high percentage of no claim for UNITED STATES however very high percentage of NOT_RECEIVED claims etc...)
We then need to add as many dummy variables as there are countries.

#### SELLER_DEPARTMENT

In [ ]:
# Creating SELLER_REGION : (we use the same department mapping as for BUYER_REGION)
for dataset in combine:
    dataset['SELLER_REGION'] = dataset['SELLER_DEPARTMENT'].map(DEPARTMENT_mapping)
    del dataset['SELLER_DEPARTMENT']

In [ ]:
# Percentage of each claim given SELLER_REGION
claim_percentage_crosstab('SELLER_REGION')

Observations :

* Unlike with BUYER_REGION where no correlation was noticeable with CLAIM_TYPE, here there are some differences between the places where the seller sends the command: commands sent from Paris (59,9% no claim) is overall more reliable than commands from Pays_De_La_Loire (34.2% no claim) for example.
Thus we need to turn BUYER_REGION into dummy variables.

* What's more, it could be unteresting to create an additionnal feature corresponding to commands sent and received in the same region (where SELLER_REGION=BUYER_REGION), since these commands may be more reliable in average.

#### PRODUCT_TYPE and PRODUCT_FAMILY

In [ ]:
# Percentage of each claim given PRODUCT_TYPE
claim_percentage_crosstab('PRODUCT_TYPE')[:40]

In [ ]:
# Percentage of each claim given PRODUCT_FAMILY
claim_percentage_crosstab('PRODUCT_FAMILY')

Observations :

* As PRODUCT_FAMILY already condensate most of information about products, we keep only a few label from PRODUCT_TYPE which stand out from the other labels are are not already in PRODUCT_FAMILY (Books in both for example). 
Among variables we can keep among PRODUCT_TYPE: PLAY CARDS (83.9% no claim), CD (60,5% no claim), COSMETIC (37.4% no claim), CELLPHONE (31,2% no claim).

* ELECTRONICS are more likely to lead to DAMAGED claims.
Indeed, electronic devices such as Television are more fragile and can be damaged during transport.
On the contrary Wine products are unlikely to be damaged, certainly due to the special care and protections set for the transport since Wine are luxury products

#### ITEM_PRICE

In [ ]:
describe(train_df['ITEM_PRICE'])

In [ ]:
# Percentage of each claim given ITEM_PRICE
claim_percentage_crosstab('ITEM_PRICE').reindex(['<10','10<20','20<50','50<100','100<500','500<1000','1000<5000','>5000'])

Observations:

* Cheap products are less likely to lead to claims.

* Pricey products are more likely to lead to WITHDRAWAL/UNDEFINED claims.

* Cheap products are more likely to lead to FAKE/NOT_RECEIVED products.

* Products in range 100<500 typically lead more to DAMAGED claims in average.
This corresponds to the price range of electronics, which are more likely to lead to DAMAGED mentions as we just saw.

## Feature engineering utilities

The following functions will be used to create new variables and remove some others.

In [ ]:
# Maps a categorical variable 'col' given a dictionnary 'mappping'.
# For example, calling this function with col = 'SHIPPING_PRICE' and 
# the mapping {"<1": 1, "1<5": 2, "5<10": 3, "10<20": 4, ">20": 5, 0.0:0} 
# will change all rows having value "<1" to 1, "1<5" to 2, etc...
def categorical_mapping(dataset, col, mapping):
    dataset[col] = dataset[col].fillna(0).map(mapping).astype(int)

# Turns a categorical variable into dummy variables.
# For example calling this with col = 'SHIPPING_MODE'
# will replace the 'SHIPPING_MODE' column with new columns 
# named 'SHIPPING_MODE_MONDIAL_RELAY_PREPAYE', 'SHIPPING_MODE_NORMAL'
# 'SHIPPING_MODE_PICKUP', 'SHIPPING_MODE_RECOMMANDE', etc...
# which values are all zero save for one.
def categorical_to_dummy(dataset, col):
    dummy = pd.get_dummies(dataset[col], prefix=col)
    for column in dummy.columns.values:
        dataset[column] = dummy[column]
    del dataset[col]

In [ ]:
# Helper function that transform both the training dataset 
# and the dataset for which we want to make prediction into
# datasets that can be used by our machine learning algorithms
# 'dataset_transform_func' is called on each dataset to perform 
# the feature engineering
# Returns:
# - train_X : the training dataset features
# - train_Y : the training dataset outputs (in categorical form)
# - train_Y_multiclass : the training dataset outputs (in dummy-encoding form)
# - testing_X : the features of the dataset for which we want to make predictions
def create_model(dataset_transform_func):
    training_df = train_df.copy()
    testing_df = test_X.copy()
    
    dataset_transform_func(training_df)
    dataset_transform_func(testing_df)
    
    #train_X = training_df.drop('CLAIM_TYPE',1).astype(int)
    train_X = training_df.drop('CLAIM_TYPE',1)
    train_Y = training_df.CLAIM_TYPE
    testing_X = testing_df.drop('ID',1)

    train_Y_multiclass = label_binarize(train_Y, classes=["-","WITHDRAWAL","DAMAGED","DIFFERENT",
                                                           "SELLER_CANCEL_POSTERIORI","NOT_RECEIVED",
                                                           "UNDEFINED","FAKE"])
    return train_X, train_Y, train_Y_multiclass, testing_X

## Model evaluation

The AUC score is used for this challenge:

In [ ]:
# Computes the ROC AUC score.
# Dummy enconding is used to represent both the predictions 
# and true values, i.e. the input are matrices and each 
# row is a vector containing a single 1 and zeros elsewhere.
def auc_weighted(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred, average='weighted')
    return score

## Model #1 (by Fabien)

This is our fist attempt. We start by preprocessing the dataset (creating new features, removing columns), then we use a RandomForest classifier (provided by scikit-learn) to predict the claims for the testing set (for which we really don't know the output).

### Data preparation

What we need to do :
- Categorical variables need to be transformed to numeric variables
- Fill missing values in variables
- Creation of new variables /  Feature Engineering

In [ ]:
def model1_feature_engineering(df):
    # Turning SHIPPING_MODE into dummy variables:
    categorical_to_dummy(df, 'SHIPPING_MODE')
    
    # Mapping SHIPPING_PRICE:
    # Considering that Null values correspond to the case when shipping is free
    SHIPPING_PRICE_mapping = {"<1": 1, "1<5": 2, "5<10": 3, "10<20": 4, ">20": 5, 0.0:0} 
    categorical_mapping(df, 'SHIPPING_PRICE', SHIPPING_PRICE_mapping)
    
    # Mapping WARRANTIES_PRICE :
    WARRANTIES_PRICE_mapping = {"<5": 1, "5<20": 2, "20<50": 3, "50<100": 4, "100<500": 5, 0.0:0}
    categorical_mapping(df, 'WARRANTIES_PRICE', WARRANTIES_PRICE_mapping)
    
    # Mapping WARRANTIES_FLG :
    WARRANTIES_FLG_mapping = {True: 1, False: 0}
    categorical_mapping(df, 'WARRANTIES_FLG', WARRANTIES_FLG_mapping)
    
    # Mapping PRICECLUB_STATUS :
    PRICECLUB_STATUS_mapping = {"UNSUBSCRIBED": 0, "REGULAR": 1, "PLATINUM": 2, "SILVER": 3, "GOLD": 4, 0.0:0}
    categorical_mapping(df, 'PRICECLUB_STATUS', PRICECLUB_STATUS_mapping)
    
    # Mapping PURCHASE_COUNT :
    PURCHASE_COUNT_mapping = {'<5':0,'5<20':1,'20<50':2,'50<100':3,'100<500':4,'>500':5}
    categorical_mapping(df, 'PURCHASE_COUNT', PURCHASE_COUNT_mapping)
    
    # Creation of UNEXPERIENCED_BUYER :
    df['UNEXPERIENCED_BUYER'] = np.where(df['PURCHASE_COUNT']==0, 1, 0)

    # Simplification of feature REGISTRATION_DATE, 
    # replaced by BUYER_SENIORITY which corresponds to the number of years the buyer is using the service
    df['BUYER_SENIORITY'] = 2017 - df['REGISTRATION_DATE']
    del df['REGISTRATION_DATE']
    
    # Simplification of feature BUYER_BIRTHDAY_DATE, replaced by BUYER_AGE
    df['BUYER_AGE'] = 2017 - df['BUYER_BIRTHDAY_DATE']
    del df['BUYER_BIRTHDAY_DATE']
        
    # Filling missing data with ffill method:
    df['BUYER_AGE'] = df['BUYER_AGE'].fillna(method='ffill').astype(int)
        
    # TODO: filling missing data with KNN
        
    # Creation of BUYER_IS_ABROAD
    df['BUYER_IS_ABROAD'] = np.where(df['BUYER_REGION']=='Etranger', 1, 0)

    # Turning BUYER_REGION into dummy_variables
    #categorical_to_dummy('BUYER_REGION')
    
    # Mapping BUYING_DATE
    # 0 for January where claims are at the top (53% claims)
    # 1 for February to November where claims are around average (around 50%)
    # 2 for October where claims are at the lowest (42% claims)
    BUYING_DATE_mapping = {1:0,10:2}
    for i in range(2,10):
        BUYING_DATE_mapping[i]=1
    categorical_mapping(df, 'BUYING_DATE', BUYING_DATE_mapping)
    
    # Mapping SELLER_SCORE_COUNT
    SELLER_SCORE_COUNT_mapping = {0.0:0, '<100':1, '100<1000':2, '1000<10000':3, '10000<100000':4, '100000<1000000':5}
    categorical_mapping(df, 'SELLER_SCORE_COUNT', SELLER_SCORE_COUNT_mapping)
    
    # Mapping SELLER_SCORE_AVERAGE
    def SELLER_SCORE_AVERAGE_map(score):
        if score < 44 or score == 50:
            return 0
        elif score < 46:
            return 1
        elif score == 46:
            return 2
        elif score == 47:
            return 3
        elif score == 48:
            return 4
        elif score == 49:
            return 5
        else:
            return 0
        
    df['SELLER_SCORE_AVERAGE'] = df['SELLER_SCORE_AVERAGE'].apply(SELLER_SCORE_AVERAGE_map)

    # Create VIP_SELLER feature for seller with a score of 49 (label 5)
    df['VIP_SELLER'] = np.where(df['SELLER_SCORE_AVERAGE']==5, 1, 0)
    
    # Turning SELLER_COUNTRY into dummy variables:
    categorical_to_dummy(df, 'SELLER_COUNTRY')
    
    # Creating SAME_REGION_BUYER_SELLER equal to 1 when Buyer and Seller are from the same region
    df['SAME_REGION_BUYER_SELLER'] = np.where(df['SELLER_REGION']==df['BUYER_REGION'], 1, 0)
    
    # Turning BUYER_REGION into dummy_variables
    categorical_to_dummy(df, 'SELLER_REGION')
    # delete the residual dummy variable linked to null values
    del df['SELLER_REGION_0']
    
    # Now we can delete BUYER_REGION
    del df['BUYER_REGION']
    
    # Turning PRODUCT_FAMILY into dummy_variables
    categorical_to_dummy(df, 'PRODUCT_FAMILY')

    # Creating dummy variables for PLAY CARDS, CD, COSMETIC and CELLPHONE:
    df['PRODUCT_TYPE_PLAY_CARDS'] = np.where(df['PRODUCT_TYPE']=='PLAY CARD', 1, 0)
    df['PRODUCT_TYPE_CD'] = np.where(df['PRODUCT_TYPE']=='CD', 1, 0)
    df['PRODUCT_TYPE_COSMETIC'] = np.where(df['PRODUCT_TYPE']=='COSMETIC', 1, 0)
    df['PRODUCT_TYPE_CELLPHONE'] = np.where(df['PRODUCT_TYPE']=='CELLPHONE', 1, 0)
    del df['PRODUCT_TYPE']
    
    # Mapping ITEM_PRICE:
    ITEM_PRICE_mapping = {"<10" : 1, "10<20" : 2, "20<50" : 3, "50<100" : 4, "100<500" : 5, "500<1000" : 6,
                          "1000<5000" : 7, ">5000" : 8} 
    categorical_mapping(df, 'ITEM_PRICE', ITEM_PRICE_mapping)
    

In [ ]:

#Create all datasets that are necessary to train, validate and test models
train_X_full, train_Y_full, train_Y_multiclass_full, testing_X = create_model(model1_feature_engineering)

In [ ]:
# Summary of features
print(train_X_full.columns.values.tolist())

In [ ]:
# Mapping train_Y:
#train_Y_full_mapping = {"-" : 0, "WITHDRAWAL" : 1, "DAMAGED" : 2, "DIFFERENT" : 3, "SELLER_CANCEL_POSTERIORI" : 4,
#                   "NOT_RECEIVED" : 5, "UNDEFINED" : 6, "FAKE" : 7} 
#train_Y_full = train_Y_full.map(train_Y_full_mapping).astype(int)

In [ ]:
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X_full, train_Y_multiclass_full, 
                                                      train_size=.7, random_state=7)

print (train_X_full.shape, train_Y_full.shape, train_X.shape, valid_X.shape, 
       train_Y.shape , valid_Y.shape , testing_X.shape)

#### MACHINE LEARNING MODELS

In [ ]:
index_Y = ["-","WITHDRAWAL","DAMAGED","DIFFERENT","SELLER_CANCEL_POSTERIORI","NOT_RECEIVED","UNDEFINED","FAKE"]

# Display the confusion matrix
def display_cm(model, train_X, train_Y, valid_X, valid_Y):
    model.fit(train_X, train_Y)
    pred_Y = model.predict(valid_X)
    cm = confusion_matrix(valid_Y, pred_Y)
    cm_df = pd.DataFrame(cm, index=index_Y, columns=index_Y)
    return cm_df

In [ ]:
# Random Forest model

random_forest = RandomForestClassifier(n_estimators=100, class_weight='balanced')
random_forest.fit(train_X, train_Y)

In [ ]:
pred_Y = random_forest.predict(valid_X)

In [ ]:
auc_weighted(valid_Y, pred_Y)

In [ ]:
# Compute ROC curve and ROC area for each class
n_classes = train_Y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(valid_Y[:, i], pred_Y[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(valid_Y.ravel(), pred_Y.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Compute macro-average ROC curve and ROC area

lw=2

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(12, 6))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()